In [66]:
import pandas as pd

In [67]:
def count_non_minus_one(row):
  return (row != -1).sum()

def longest_consecutive_streak(row):
  current_streak = 0
  longest_streak = 0

  for num in row:
    if num == 0:
      longest_streak = max(longest_streak, current_streak)
      current_streak = 0
    elif num != -1:
      current_streak += 1
      
  longest_streak = max(longest_streak, current_streak)
  return longest_streak

def longest_consecutive_perfect_streak(row):
  current_streak = 0
  longest_streak = 0

  for num in row:
    if num == 0 or num == 4 or num == 5:
      longest_streak = max(longest_streak, current_streak)
      current_streak = 0
    elif num != -1:
      current_streak += 1

  longest_streak = max(longest_streak, current_streak)
  return longest_streak

def longest_fail_streak(row):
  current_streak = 0
  longest_streak = 0

  for num in row:
    if num != -1 and num != 4 and num != 5 and num != 8:
      current_streak += 1
    else:
      longest_streak = max(longest_streak, current_streak)
      current_streak = 0

  longest_streak = max(longest_streak, current_streak)
  return longest_streak

def get_score_average_for_player_from_matchday_list(score_dataframe: pd.DataFrame, matchday_dictionary: dict, team_name: str, player_name: str):
    list_of_matchdays = matchday_dictionary.get(team_name)
    selected_values = score_dataframe.loc[list_of_matchdays, player_name]
    list_of_selected_values = selected_values.tolist()
    values_to_drop = [-1]
    filtered_list = [value for value in list_of_selected_values if value not in values_to_drop]
    average_guess = round(sum(filtered_list) / len(filtered_list), 2) if len(filtered_list) != 0 else 0
    return average_guess

In [68]:
initial_csv = pd.read_csv('predictions.csv')
full_df = initial_csv.set_index('name')
result = full_df.where(full_df != -1).sum(axis=1)

In [69]:
only_md_df = full_df.loc[:, ~full_df.columns.str.startswith('b')]
matchday_df = only_md_df.copy()
matchday_df['counter'] = matchday_df.apply(count_non_minus_one, axis=1)
matchday_df['points_getting_counter'] = ((only_md_df != -1) & (only_md_df != 0)).sum(axis=1)
matchday_df['correct_winner_percentage'] = round(matchday_df['points_getting_counter'] / matchday_df['counter'] * 100, 1)
matchday_df['matchday_sum'] = only_md_df.where(only_md_df != -1).sum(axis=1)
matchday_df['points_per_matchday'] = round(matchday_df['matchday_sum'] / matchday_df['counter'], 3)
matchday_df['perfect_guesses'] = only_md_df.eq(8).sum(axis=1)
matchday_df['longest_streak'] = only_md_df.apply(longest_consecutive_streak, axis=1)
matchday_df['longest_perfect_streak'] = only_md_df.apply(longest_consecutive_perfect_streak, axis=1)
matchday_df['longest_fail_streak'] = only_md_df.apply(longest_fail_streak, axis=1)

In [70]:
sorted_df = matchday_df.sort_values(by='points_per_matchday', ascending=False)
sorted_df.head()

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,m51,counter,points_getting_counter,correct_winner_percentage,matchday_sum,points_per_matchday,perfect_guesses,longest_streak,longest_perfect_streak,longest_fail_streak
name,,,,,,,,,,,,,,,,,,,,,
Adomas K,4,0,4,4,5,5,5,0,0,5,...,4,51,27,52.9,149.0,2.922,8,5,2,7
Modestas Čižius,4,4,0,4,4,0,4,0,0,4,...,4,51,26,51.0,139.0,2.725,7,3,2,5
EmilisT,4,5,4,5,0,8,4,0,0,4,...,5,51,25,49.0,134.0,2.627,7,4,1,7
Edvinas Lašinskas,4,4,4,4,4,0,4,0,0,5,...,8,49,24,49.0,124.0,2.531,5,5,2,4
Gelezinis Vilkas,0,4,0,8,8,0,5,4,0,4,...,4,51,25,49.0,125.0,2.451,4,4,2,6


In [72]:
t_matchday_df = only_md_df.copy().T
md_df = t_matchday_df.copy()
md_df.index.name ='matchday'
md_df['counter'] = t_matchday_df.apply(count_non_minus_one, axis=1)
md_df['matchday_sum'] = t_matchday_df.where(t_matchday_df != -1).sum(axis=1)
md_df['average_points'] = round(md_df['matchday_sum'] / md_df['counter'], 3)

In [73]:
games_csv = pd.read_csv('games.csv')
games_df = games_csv.set_index('matchday')
games_df_no_round = games_df.drop('round', axis=1)

In [74]:
joined_df = pd.merge(md_df, games_df, on="matchday").drop(['counter', 'matchday_sum', 'average_points', 'round'], axis=1)

In [75]:
joined_df.head()

,Adomas K,Adrianas,Buzas,Deivis7,Dominykas Žvirblis,Edvinas Lašinskas,Edvinas Pašvenskas,EmilisT,Ernest Moroz,Gelezinis Vilkas,Gustelio36,Modestas Čižius,Rimvydas Peslekas,Rolandas Moscian,Romuald1993 Lipniaszka,Sarunas Beivydas,Tadas Laukaitis,Vold Uss,team_1,team_2
matchday,,,,,,,,,,,,,,,,,,,,
m1,4,-1,-1,4,4,4,0,4,-1,0,4,4,4,4,-1,4,4,4,ger,sco
m2,0,5,0,4,0,4,4,5,0,4,4,4,4,4,-1,-1,0,0,hun,swi
m3,4,0,4,4,4,4,0,4,4,0,4,0,4,4,-1,0,4,0,esp,cro
m4,4,0,4,4,4,4,4,5,4,8,4,4,4,4,-1,4,4,4,ita,alb
m5,5,4,8,8,0,4,4,0,8,8,4,4,4,4,-1,4,8,0,pol,ned


In [76]:
team_matchdays = {}

for idx, row in games_df_no_round.iterrows():
  for value in row:
    if value in team_matchdays:
      team_matchdays[value].append(idx)
    else:
      team_matchdays[value] = [idx]

In [77]:
def get_score_average_for_player_from_matchday_list(score_dataframe: pd.DataFrame, matchday_dictionary: dict, team_name: str, player_name: str):
    list_of_matchdays = matchday_dictionary.get(team_name)
    selected_values = score_dataframe.loc[list_of_matchdays, player_name]
    list_of_selected_values = selected_values.tolist()
    values_to_drop = [-1]
    filtered_list = [value for value in list_of_selected_values if value not in values_to_drop]
    average_guess = round(sum(filtered_list) / len(filtered_list), 2) if len(filtered_list) != 0 else 0
    return average_guess

def get_score_average_for_team(team_name: str, score_dataframe: pd.DataFrame = t_matchday_df, matchday_dictionary: dict = team_matchdays):
    list_of_matchdays = matchday_dictionary.get(team_name)
    selected_values = score_dataframe.loc[list_of_matchdays]
    all_values_list = selected_values.stack().tolist()
    filtered_values_list = [value for value in all_values_list if value != -1]
    average_score = sum(filtered_values_list) / len(filtered_values_list)
    return round(average_score, 3)

In [78]:
team_player_averages = pd.DataFrame(columns=t_matchday_df.columns, index=team_matchdays.keys())

for team_name in team_matchdays.keys():
  for player_name in t_matchday_df.columns:
    average = get_score_average_for_player_from_matchday_list(t_matchday_df, team_matchdays, team_name, player_name)
    team_player_averages.loc[team_name, player_name] = average

team_player_averages['average'] = team_player_averages.apply(lambda row: get_score_average_for_team(row.name), axis=1)
team_player_averages = team_player_averages.sort_values(by='average', ascending=False)

In [79]:
team_player_averages.sort_values(by='average', ascending=True).head()

name,Adomas K,Adrianas,Buzas,Deivis7,Dominykas Žvirblis,Edvinas Lašinskas,Edvinas Pašvenskas,EmilisT,Ernest Moroz,Gelezinis Vilkas,Gustelio36,Modestas Čižius,Rimvydas Peslekas,Rolandas Moscian,Romuald1993 Lipniaszka,Sarunas Beivydas,Tadas Laukaitis,Vold Uss,average
ukr,0.0,2.67,0.0,0.0,1.67,0.0,0.0,1.33,0.0,1.33,0.0,0.0,0.0,2.5,0.0,2.67,0.0,2.67,0.808
svn,4.5,2.0,0.0,0.0,2.0,0.0,1.25,2.0,0.0,1.25,0.0,0.0,0.0,0.0,0.0,1.25,0.0,3.25,0.986
swi,1.6,1.67,0.8,0.8,0.0,2.4,1.8,1.0,3.0,3.4,1.0,4.0,0.8,1.0,0.0,1.25,0.0,0.8,1.444
svk,3.25,2.0,2.0,2.0,3.33,1.25,2.0,1.0,0.0,0.0,0.0,1.25,2.0,2.5,0.0,2.0,2.0,2.0,1.586
aut,1.25,1.0,1.0,1.0,2.0,2.25,1.0,1.0,1.33,2.0,1.0,2.25,2.0,1.0,5.0,1.0,1.0,2.25,1.634


In [80]:
def cumulative_columns(df: pd.DataFrame):
  cumulative_df = pd.DataFrame(index=df.index)
  for i in range(df.shape[1]):
      cumulative_df[f'm{i+1}'] = df.iloc[:, :i+1].replace(-1, 0).sum(axis=1)
  return cumulative_df

cumulative_scores_small = cumulative_columns(only_md_df)

In [81]:
cumulative_scores_small.head()

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,m42,m43,m44,m45,m46,m47,m48,m49,m50,m51
name,,,,,,,,,,,,,,,,,,,,,
Adomas K,4,4,8,12,17,22,27,27,27,32,...,116,116,116,124,129,129,137,145,145,149
Adrianas,0,5,5,5,9,9,13,13,13,17,...,87,91,91,91,91,91,91,91,91,91
Buzas,0,0,4,8,16,16,20,20,20,24,...,81,85,85,85,85,85,93,93,101,109
Deivis7,4,8,12,16,24,24,28,28,28,32,...,96,100,100,100,100,100,105,105,105,105
Dominykas Žvirblis,4,4,8,12,12,20,25,25,25,29,...,92,96,96,96,96,96,96,96,96,96


In [82]:
def cumulative_sums_with_renamed_bonus(df, bonus_positions):
  columns = sorted(df.columns)

  for pos, col in enumerate(columns):
      if col.startswith('b'):
          new_col_name = f'm{bonus_positions[pos - 1]}_'
          df.rename(columns={col: new_col_name}, inplace=True)

  df = df.replace(-1, 0)
  cumulative_df = df.cumsum(axis=1)

  return cumulative_df

In [83]:
bonus_positions = [37, 46, 51, 54, 56]
cumulative_scores = cumulative_sums_with_renamed_bonus(full_df, bonus_positions)


In [84]:
cumulative_scores.head()

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,m45,m46,m47,m48,m46_,m49,m50,m51_,m51,m54_
name,,,,,,,,,,,,,,,,,,,,,
Adomas K,4,4,8,12,17,22,27,27,27,32,...,130,135,135,143,143,151,151,151,155,155
Adrianas,0,5,5,5,9,9,13,13,13,17,...,91,91,91,91,91,91,91,91,91,91
Buzas,0,0,4,8,16,16,20,20,20,24,...,85,85,85,93,93,93,101,101,109,109
Deivis7,4,8,12,16,24,24,28,28,28,32,...,106,106,106,111,117,117,117,125,125,125
Dominykas Žvirblis,4,4,8,12,12,20,25,25,25,29,...,102,102,102,102,102,102,102,102,102,102


In [85]:
def rename_columns(df_scores, df_matches):
  new_columns = {}
  for col in df_scores.columns:
      if not col.endswith('_'):
          new_col = f"{df_matches['team_1'].loc[col].upper()} - {df_matches['team_2'].loc[col].upper()}"
          new_columns[col] = new_col
      else:
          new_columns[col] = col

  df_scores = df_scores.rename(columns=new_columns)
  return df_scores

df_scores = rename_columns(cumulative_scores, games_df_no_round)
df_scores_small = rename_columns(cumulative_scores_small, games_df_no_round)

In [86]:
def rename_bonus_columns(df):
  rename_dict = {
      'm56_': 'GROUP BONUS',
      'm37_': '1/8 BONUS',
      'm46_': 'QUARTER-FINAL BONUS',
      'm51_': 'SEMI-FINAL BONUS',
      'm54_': 'FINAL BONUS'
  }
  df = df.rename(columns=rename_dict)
  return df

df_scores_normal_names = rename_bonus_columns(df_scores)

In [87]:
df_scores_normal_names.head()

,GER - SCO,HUN - SWI,ESP - CRO,ITA - ALB,POL - NED,SVN - DEN,SER - ENG,ROM - UKR,BEL - SVK,AUT - FRA,...,ESP - GER,POR - FRA,ENG - SWI,NED - TUR,QUARTER-FINAL BONUS,ESP - FRA,NED - ENG,SEMI-FINAL BONUS,ESP - ENG,FINAL BONUS
name,,,,,,,,,,,,,,,,,,,,,
Adomas K,4,4,8,12,17,22,27,27,27,32,...,130,135,135,143,143,151,151,151,155,155
Adrianas,0,5,5,5,9,9,13,13,13,17,...,91,91,91,91,91,91,91,91,91,91
Buzas,0,0,4,8,16,16,20,20,20,24,...,85,85,85,93,93,93,101,101,109,109
Deivis7,4,8,12,16,24,24,28,28,28,32,...,106,106,106,111,117,117,117,125,125,125
Dominykas Žvirblis,4,4,8,12,12,20,25,25,25,29,...,102,102,102,102,102,102,102,102,102,102


In [88]:
# save if necessacry

# df_scores_normal_names.to_csv('final_scores.csv')
# df_scores_small.to_csv('no_bonus_scores.csv')

In [89]:
import pandas as pd

def process_dataframe(df_1, df_2, df_3):
  results = []
  for index, row in df_1.iterrows():
    df_2['index'] = index
    df_2 = df_2.sort_values(by=index, ascending=False)
    top_three = df_2[index].head(3)
    bottom_three = df_2[index].tail(3)
    m54_value = df_3.loc[index, 'm54_']
    result = {
        'name': index,
        'counter': int(row['counter']),
        'correct_winner_percentage': row['correct_winner_percentage'],
        'points_per_matchday': row['points_per_matchday'],
        'total_points': m54_value,
        'matchday_points': int(row['matchday_sum']),
        'perfect_guesses': int(row['perfect_guesses']),
        'longest_streak': int(row['longest_streak']),
        'longest_fail_streak': int(row['longest_fail_streak']),
        'top_three': top_three.to_dict(),
        'bottom_three': bottom_three.to_dict()
    }
    results.append(result)

  return pd.DataFrame(results)


In [90]:
processed_df = process_dataframe(sorted_df, team_player_averages, cumulative_scores)

In [96]:
processed_df.iloc[0]

name                                                       Adomas K
counter                                                          51
correct_winner_percentage                                      52.9
points_per_matchday                                           2.922
total_points                                                    155
matchday_points                                                 149
perfect_guesses                                                   8
longest_streak                                                    5
longest_fail_streak                                               7
top_three                     {'ger': 5.6, 'esp': 4.71, 'svn': 4.5}
bottom_three                 {'czk': 1.33, 'aut': 1.25, 'ukr': 0.0}
Name: 0, dtype: object

In [92]:
# Totals

total_counter = sorted_df['counter'].sum()
pt_counter = sorted_df['points_getting_counter'].sum()
total_md_points = sorted_df['matchday_sum'].sum()
total_points_per_md = round(total_md_points / total_counter, 3)
total_points = cumulative_scores['m54_'].sum()
perfect_guesses = sorted_df['perfect_guesses'].sum()
print(f'Counter: {total_counter}, points getting pctg: {round(pt_counter / total_counter * 100, 2)}, points per md: {total_points_per_md}')
print(f'Md points: {int(total_md_points)}, total points: {total_points}, perfect guesses: {perfect_guesses}')

Counter: 863, points getting pctg: 45.65, points per md: 2.336
Md points: 2016, total points: 2188, perfect guesses: 86


In [93]:
T_md_df = only_md_df.T

In [94]:
corr_team_df = team_player_averages.drop(['index', 'average'], axis=1)

In [95]:
import numpy as np

def get_most_similar(df):
    correlation_matrix = df.corr()
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    highest_correlation = upper_triangle.max().max()
    max_index = upper_triangle.stack().idxmax()
    row_label, col_label = max_index
    print("Highest correlation between:", row_label, "and", col_label, " (value of ", highest_correlation, ")")

get_most_similar(corr_team_df)
get_most_similar(T_md_df)

Highest correlation between: Deivis7 and Rolandas Moscian  (value of  0.7361915477305984 )
Highest correlation between: Deivis7 and Rolandas Moscian  (value of  0.7336452976191022 )


Link to Canva visualizations: https://www.canva.com/design/DAGLyhwtyYY/GJ63q0KpWdn5kNv_SEl9Zw/edit?utm_content=DAGLyhwtyYY&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton